# Application Prototyping
Use this notebook to test the components of the TeXRay module. 

In [18]:
# import the ray class
sys.path.append("c:\\Users\\Dave Semeraro\\Documents\\TeXRay")
from txr.render import rays, image
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
import numpy as np  

## image module tests. 
Test the image module bits below. 

In [ ]:
# instantiate an image object
tricolore = image.image(height = 300, width = 200)
height = tricolore.height
width = tricolore.width
print(f'Image height: {tricolore.height} Image width: {tricolore.width} ')
# load the italian flag. Drape vertically green on top
red = (255,0,0)
white = (255,255,255)
green = (0,255,0)
for row in range(height):
    for col in range(width):
        if row < height/3:
            tricolore[row,col] = red
        elif row < 2*height/3:
            tricolore[row,col] = white
        else:
            tricolore[row,col] = green

In [ ]:
# display the tricolore
plt.imshow(Image.fromarray(tricolore[:,:]))

## Ray module tests
Ray module tests go here. 
### Ray generator and ray group
The ray generator expects origin and direction data in a format that can be parsed for three tuples. Try a list of tuples first.

In [ ]:
# test the raygenerator base class
# going to need a couple of origins and directions. 
origins = [(0.,0.,0.),(0.1,0.,0.,)]
directions = [(1.,1.,0.),(1.,0.,1.)]
# base ray generator
raygen = rays.ray_generator(origins,directions)
# use the ray generator to make rays
raygen.generate()
# get the ray group container from the generator
rayGroup = raygen.ray_group
# show the contents of the ray group

In [ ]:
# dump the ray data
rayGroup.rays

The results of the cell above should be an array with two rows corresponding to the two origins and directions. 

Now try single origin in a list. 

In [ ]:
origin = [(5.,8.,2)]
raygen = rays.ray_generator(origin,directions)
raygen.generate()
rayGroup = raygen.ray_group
rayGroup.rays

The output of the cell above should be two rays with the same origin and different directions. 
#### Numpy arrays
Now let's try using numpy arrays to hold the origin and direction data. First try passing arrays with shape 2,3. Two rows of three elements each. 

In [ ]:
origins = np.array([[0.,0.,0.],[5.0,8.0,2.0]])
directions = np.array([[1.0,1.0,0.0],[1.0,0.0,1.0]])
raygen = rays.ray_generator(origins,directions)
raygen.generate()
rayGroup = raygen.ray_group
rayGroup.rays

Now lets try an array of tuples

In [ ]:
origins = np.array([(0.,1.,0.),(5.0,8.0,2.0)])
directions = np.array([(1.0,1.0,0.0),(1.0,0.0,1.0)])
raygen = rays.ray_generator(origins,directions)
raygen.generate()
rayGroup = raygen.ray_group
rayGroup.rays

How about a tuple of tuples


In [19]:
origins = ((0.,1.,0.),(5.0,8.0,2.0))
directions = ((1.0,1.0,0.0),(1.0,0.0,1.0))
raygen = rays.ray_generator(origins,directions)
raygen.generate()
rayGroup = raygen.ray_group
rayGroup.rays

array([[0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 1.1754944e-38, 3.4028235e+38,
        1.1754944e-38, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [5.0000000e+00, 8.0000000e+00, 2.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 1.0000000e+00, 1.1754944e-38, 3.4028235e+38,
        1.1754944e-38, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],
      dtype=float32)

Now multiple origin and single directions

In [ ]:
origins = np.array([(0.,0.,0.),(0.,1.,0.)])
directions = np.array([(1.0,1.0,0.0)])
raygen = rays.ray_generator(origins,directions)
raygen.generate()
rayGroup = raygen.ray_group
rayGroup.rays

Looks like ray generation is going well. Lets see about pulling a single ray from the set. 

In [ ]:
raydata = rayGroup[0]
raydata

In [ ]:
raydata = rayGroup[0:2,0:6]
raydata

In [ ]:
origin = rayGroup[0,0:3]
direction = rayGroup[0,3:6]
print(f'{origin} {direction}')

Lets see what happens to the data in raygroup when we build a ray from the raydata row, and change the value of the payload of that ray.

In [21]:
print(f'original raygroup data')
print(rayGroup[:])
# grab the first ray
raydata = rayGroup[0]
# get the data needed to generate a ray
origin = raydata[0:3]
direction = raydata[3:6]
t = raydata[8]
# make a new ray object
r = rays.ray(origin,direction,t)
# set the payload in the new ray to be the payload from the original
r.payload = raydata[9:12]
print(f'data in raygroup row 0 {raydata}')
r.payload = (4,5,60)
print(f'data in new ray after payload set {r.raydata}')
# write the payload back to the data in raygroup
print(f'data in raydata before r overwrites it {raydata}')
raydata = r.raydata
print(f'data in raydata after ray overwrites it {raydata}')
print(f'data in the raygroup after ray overwrites raydata')
print(rayGroup[:])
rayGroup[0] = r.raydata
print(f'data in the raygroup after ray overwrites rayGroup[0]')
print(rayGroup[:])

original raygroup data
[[0.0000000e+00 1.0000000e+00 0.0000000e+00 1.0000000e+00 1.0000000e+00
  0.0000000e+00 1.1754944e-38 3.4028235e+38 1.1754944e-38 4.0000000e+00
  5.0000000e+00 6.0000000e+01]
 [5.0000000e+00 8.0000000e+00 2.0000000e+00 1.0000000e+00 0.0000000e+00
  1.0000000e+00 1.1754944e-38 3.4028235e+38 1.1754944e-38 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
data in raygroup row 0 [0.0000000e+00 1.0000000e+00 0.0000000e+00 1.0000000e+00 1.0000000e+00
 0.0000000e+00 1.1754944e-38 3.4028235e+38 1.1754944e-38 4.0000000e+00
 5.0000000e+00 6.0000000e+01]
data in new ray after payload set [0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
 1.00000000e+00 0.00000000e+00 1.17549435e-38 3.40282347e+38
 1.17549435e-38 4.00000000e+00 5.00000000e+00 6.00000000e+01]
data in raydata before r overwrites it [0.0000000e+00 1.0000000e+00 0.0000000e+00 1.0000000e+00 1.0000000e+00
 0.0000000e+00 1.1754944e-38 3.4028235e+38 1.1754944e-38 4.0000000e+00
 5.0000000e+00 6.0000000e+01]
da

In [23]:
rayGroup[0:2,9:12].shape

(2, 3)